# [Structural Pattern Matching](https://www.python.org/dev/peps/pep-0634/)

In [7]:
!pip install black[jupyter]

     |████████████████████████████████| 155 kB 4.2 MB/s eta 0:00:01
  Using cached click-8.0.3-py3-none-any.whl (97 kB)
  Using cached pathspec-0.9.0-py2.py3-none-any.whl (31 kB)
  Using cached mypy_extensions-0.4.3-py2.py3-none-any.whl (4.5 kB)
     |████████████████████████████████| 288 kB 8.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/cdevlin/v/ve/p310/bin/python3.10 -m pip install --upgrade pip' command.


# Old and busted

In [2]:
def ext(lang):
    if lang == "Python":
        return ".py"
    elif lang == "Ruby":
        return ".rb"
    elif lang == "Perl":
        return ".pl"
        case "Perl 6":
        return ".p6"
    elif lang in ("JavaScript", "ECMAScript"):
        return ".js"
    else:
        return "Language not welcome at DDL"

In [4]:
ext("Ruby")

'.rb'

# Alternately

In [5]:
def ext(lang):
    return {
        "Python": ".py",
        "Ruby": ".rb",
        "Perl": ".pl",
        "Perl 6": ".p6",
        "JavaScript": ".js",
        "ECMAScript": ".js",
    }.get(lang, "Language not welcome at DDL")

In [6]:
ext("Java")

'Language not welcome at DDL'

# New Hotness

In [2]:
def ext(lang):
    match lang:
        case "Python":
            return ".py"
        case "Ruby":
            return ".rb"
        case "Perl":
            return ".pl"
        case "Perl 6":
            return ".p6"
        case ("JavaScript", "ECMAScript"):
            return ".js"
        case _:
            return "Language not welcome at DDL"

In [3]:
ext("ECMAScript")

'Language not welcome at DDL'

### Oops!

That wasn't actually the right syntax for `"JavaScript" | "ECMAScript"`

In [6]:
def ext(lang):
    match lang:
        case "Python":
            return ".py"
        case "Ruby":
            return ".rb"
        case "Perl":
            return ".pl"
        case "Perl 6":
            return ".p6"
        case "JavaScript" | "ECMAScript":
            return ".js"
        case _:
            return "Language not welcome at DDL"

In [8]:
ext('ECMAScript')

'.js'

# Bored yet?

Annoyed?  Offended?

In [118]:
def fn(args):
    match args:
        case 1:
            return 'one'
        case 2:
            return 'two'
        case x:  # this is "name binding"
            return f'number {x}'

In [81]:
fn(6)

'number 6'

Matching the *structure*, and *maybe* also matching constant values

In [20]:
import math

In [3]:
def area(*dimensions):
    match dimensions:
        case [0]:
            return "A point; zero volume"   
        case [l] if l >= 0:  # this is a "guard"
            return f"Square {l} x {l}, area {l**2}"
        case [l]:
            return "No negative dimensions, please"
        case [l, w]:  
            return f"Rectangle {l} x {w}, area {l * w}"             
        case [l, w, h]:
            return f"Prism {l} x {w} x {h}, volume {l * w * h}"
        case [*x]:
            tot = math.prod(x)
            descriptor = ' x '.join(str(y) for y in x)
            return f"{len(x)}-dimensional object {descriptor} = {tot}"
        case _:
            return "I only operate on lists"

In [4]:
area(0)

'A point; zero volume'

### Dict matching

Note extra keys ignored

In [5]:
def classified(itm):
    match itm:
        case {"name": n, "class": "ship", "crew": crew}:
            return f"{n}; crew {crew}"
        case {"name": n, "class": "planet", "diameter, km": diam} if diam > 10_000:
            return f"large planet {n}"      
        case {"name": n, "class": "planet", "diameter, km": diam}:
            return f"small planet {n}"    
        case {"name": n, "class": "droid"}:  
            return f"droid {n}"        
        case _:
            return "oops"

In [6]:
things = [
    {"name": "Rogue One", "class": "ship", "type": "Zeta-class Heavy Cargo Shuttle", "crew": 2, },
    {"name": "Lah'mu", "class": "planet", "diameter, km": 12_618},    
    {"name": "K2SO", "class": "droid", "type": "KX-series security droid"},
    {"name": "Scarif", "class": "planet", "diameter, km": 9112},    
    {"name": "Eadu", "class": "planet", "diameter, km": 14_121},    
]
for itm in things:
    print(classified(itm))

Rogue One; crew 2
large planet Lah'mu
droid K2SO
small planet Scarif
large planet Eadu


## Object matching


In [9]:
from dataclasses import dataclass

@dataclass
class Ship:
    name: str
    type_: str 
    crew: int
    
@dataclass 
class Planet:
    name: str
    diam_km: float
    
@dataclass
class Droid:
    name: str
    model: str

In [10]:
def classified(itm):
    match itm:
        case Ship(s):
            return f"{s.name}; crew {crew.int}"


In [11]:
classified(Ship(name="Rogue One", type_="Zeta-class Heavy Cargo Shuttle", crew=2))

AttributeError: 'str' object has no attribute 'name'

In [66]:
def season(dt):
    match (dt.day, dt.month, dt.year):
        case (day, 12, year) | (day, 1, year) | (day, 2, year):
            print(f'Winter of {year}')
        case (day, 3, year) | (day, 4, year) | (day, 5, year):
            print(f'Spring of {year}')
        case (day, 6, year) | (day, 7, year) | (day, 8, year):
            print(f'Summer of {year}')
        case (day, _, year):
            print(f'Fall of {year}')
        case _:
            print('Invalid data')



In [67]:
from datetime import date
season(date(1932,4,1))

Spring of 1932


In [75]:
def sql(command):
    match command.lower().split():
        case ["select", col, "from", tbl]:
            return {"action": "select", "target": f"{tbl}.{col}"}
        case ["insert", "into", tbl, "values", *val_list]:
            return {"action": "insert", "target": tbl, "values": val_list}

In [76]:
sql("SELECT crew FROM ship")

{'action': 'select', 'target': 'ship.crew'}

In [77]:
sql("insert into ship values ('Rogue One', 'Millenium Falcon')")

{'action': 'insert',
 'target': 'ship',
 'values': ["('rogue", "one',", "'millenium", "falcon')"]}

### doesn't work

- Match against regexes
- Match agains ranges

Use guards (which is kind of like going back to if statements after all)

In [115]:
import re
def matcher(name):
    match name:
        case re.compile("[\A\-]{3,5}$"):
            return "droid"
        case _:
            return "not a droid"

In [116]:
matcher("chewie")

TypeError: called match pattern must be a type